## Call ArcGIS API for Python and publish CSV

In [ ]:
from arcgis.gis import GIS

In [ ]:
gis = GIS("https://www.arcgis.com", "my_username")

In [ ]:
csv_file = "Air Quality Monitors.csv"
csv_item = gis.content.add({}, csv_file)
csv_item

In [ ]:
csv_lyr = csv_item.publish()
csv_lyr

In [ ]:
map = gis.map("Air Quality Monitors")
display(map)

In [ ]:
map.add_layer(csv_lyr)
map.extent = csv_lyr.extent

## Update CSV

In [ ]:
from arcgis.features import FeatureLayerCollection
csv_lyr = gis.content.search('title:"Air Quality Monitors" AND owner:"my_username"', 'Feature Layer')[0]
flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
flayer_collection.manager.overwrite(r'\path\to\updated_data\Air Quality Monitors.csv')

In [ ]:
map.remove_layers()
map.add_layer(csv_lyr)

## Creating an Insights Workbook using REST

In [ ]:
import json
import random
# Workbook title
title = "Script-Created Workbook"
# Workspace ID (8 random hex digits)
workbook_id = '%08x' % random.randrange(16**8)
# Hosted service URL differs between Portal and AGOL
insights_service_url = gis._url.lower() + \
    '/arcgis/rest/services/Hosted/'
if gis._url.find('arcgis.com') >= 0:
    insights_service_url = 'https://insightsservices.arcgis.com/' + gis._portal._properties['id'] + '/arcgis/rest/services/'
workspace_url = insights_service_url + workbook_id + "/WorkspaceServer"

# Path for REST API to create new Workspace Service (ArcGIS Insights internal data storage)
path = gis._portal.url + '/sharing/rest/content/users/' + \
    gis.users.me.username + '/createService'
post_data = {'f': 'json',
             'createParameters': '{"name": "' + workbook_id + '"}',
             'targetType': 'workspaceService'}
# The first call creates the workspace, but the Workbook is not
# functional until after the second POST below
resp = gis._portal.con.post(path, post_data)
# Depending on whether it's on Portal or AGOL, it will use a
# different key for ID.
if gis._url.find('arcgis.com') < 0:
    workspace_id = resp['itemId']
else:
    workspace_id = resp['serviceItemId']

item_props = {
    "f": "json",
    "id": workspace_id,
    "type": "Insights Workbook",
    "title": title}
# For some reason this is not needed for Portal, but it is for AGOL
if gis._url.find('arcgis.com') >= 0:
    item_props["url"] = workspace_url

# Full set of default JSON data properties for an ArcGIS Insights
# workbook item.
props = {
    "format": 9,
    "title": title,
    "pages": [{
        "title": "Page 1",
        "model": {
            "items": []
        },
        "cards": [],
        "layout": [],
        "contents": []
    }],
    "activePage": 0,
    "workspace": {
        "datasets": {}
    },
    "_ssl": True,
    "created": 0,
    "modified": 0,
    "guid": None,
    "type": "Insights Workbook",
    "typeKeywords": ["Application", "ArcGIS",
                     "Insights Workbook", "Hosted Service"],
    "description": None,
    "tags": [],
    "snippet": None,
    "thumbnail": None,
    "documentation": None,
    "extent": [],
    "categories": [],
    "spatialReference": None,
    "accessInformation": None,
    "licenseInfo": None,
    "culture": "english (united states)",
    "properties": None,
    "proxyFilter": None,
    "access": "private",
    "size": 0,
    "appCategories": [],
    "industries": [],
    "languages": [],
    "largeThumbnail": None,
    "banner": None,
    "screenshots": [],
    "listed": False,
    "ownerFolder": None,
    "protected": False,
    "commentsEnabled": True,
    "numComments": 0,
    "numRatings": 0,
    "avgRating": 0,
    "numViews": 3,
    "itemControl": "admin",
    "scoreCompleteness": 0,
    "groupDesignations": None
}
# After the first call sets up the workspace, this second call sets
# up the actual Workbook with all the data props, title, etc.
gis._portal.update_item(workspace_id, item_props, json.dumps(props))

## Adding a feature layer

In [ ]:
lyr = gis.content.search('title:"Air Quality Monitors" AND owner:"my_username"', 'Feature Layer')[0]
lyr

In [ ]:
 # Random 7-digit hex suffix for dataset name
dataset_name = lyr.title + '_%07x' % random.randrange(16**7)
post_data = {'f': 'json',
             'tools': '[{"name":"add-data","params":{"data":{' +
                      '"type":"feature-layer",' +
                      '"url":"' + lyr.url + '/0'  +
                      '"}},"outDataset":"' + dataset_name + '"}]',
             'outDatasets': '["' + dataset_name + '"]'}
# Execute the add-data operation within ArcGIS Insights. Note: data
# is not automatically saved in the Workbook. Must manually call
# save in order to make it permanent.
resp = gis._portal.con.post(workspace_url + '/execute', post_data)

# In addition to calling execute to create the dataset, it must also
# be placed in the Workbook Item properties in several places:
props['pages'][0]['model']['items'].append({
    'operation': 'add-data',
    'params': {
        'data': {
            'type': 'feature-layer',
            'url': lyr.url + '/0'
        }
    },
    'outDataset': dataset_name
})
props['pages'][0]['contents'].append({
    'dataset': dataset_name
})
# Get extent
my_extent = json.loads(lyr.layers[0].properties.extent.__repr__())
# resp[dataset_name] refers to internal data ID
props['workspace']['datasets'][dataset_name] = {
    'data': resp[dataset_name],
    'owner': lyr.id,
    'fields': {
        'shape': {
            'alias': 'Location'
        }
    },
    'extent': my_extent,
    'origin': True
}

## Save workbook

In [ ]:
# A few properties have to be manually set at save (doesn't work to
# just set them on initial Workbook creation).
props["id"] = workspace_id
props["owner"] = gis.users.me.username
props["name"] = workbook_id
props["url"] = workspace_url
post_data = {
    'f': 'json',
    'title': title,
    'text': json.dumps(props)}
# Basically just a standard ArcGIS item update with the updated JSON
# properties
update_url = gis._url.lower() + '/sharing/rest/content/users/' + \
    gis.users.me.username + '/items/' + \
    workspace_id + '/update'
gis._portal.con.post(update_url, post_data)

## Add map

In [ ]:
# Add map, with layer and extent
props['pages'][0]['cards'].append({
    'type': 'map',
    'title': 'Card 1',
    'content': {
        'layers': [{'datasetId': dataset_name}],
        'extent': my_extent
    }
})
# place map on layout
props['pages'][0]['layout'].append({
    "x": 0,
    "y": 0,
    "w": 20,
    "h": 20
})

In [ ]:
# Save again
props["id"] = workspace_id
props["owner"] = gis.users.me.username
props["name"] = workbook_id
props["url"] = workspace_url
post_data = {
    'f': 'json',
    'title': title,
    'text': json.dumps(props)}
update_url = gis._url.lower() + '/sharing/rest/content/users/' + \
    gis.users.me.username + '/items/' + \
    workspace_id + '/update'
gis._portal.con.post(update_url, post_data)

## Add chart

In [ ]:
in_dataset_base = dataset_name[:dataset_name.find('_')]
# Generate new dataset name with 7 random hex digits as suffix
out_dataset = in_dataset_base + '_%07x' % random.randrange(16**7)
# Perform aggregate operation
props['pages'][0]['model']['items'].append({
    'operation': 'aggregate',
    'params': {
        'dataset': dataset_name,
        'groupBy': ["EPARegion"],
        'statistics': [{
            'type': "count",
            'field': "ObjectId",
            'outField': "objectid_count"
        }],
        'totals': False
    },
    'outDataset': out_dataset
})
in_data_id = props['workspace']['datasets'][dataset_name]['data']
# Add new dataset to workspace
props['workspace']['datasets'][out_dataset] = {
    'data': {
        'metadata': {
            'fields': [
                {
                    'name': "EPARegion",
                    'alias': "EPARegion",
                    'type': "esriFieldTypeString",
                    'entity': 'e0'
                },
                {
                    'name': "objectid_count",
                    'alias': "objectid_count",
                    'type': "esriFieldTypeInteger"
                }
            ],
            'entities': [{
                'fields': [
                    "EPARegion",
                    "objectid_count"
                ],
                'keyFields': ["EPARegion"]
            }]
        },
        'tools': [{
            'name': 'aggregate',
            'params': {
                'dataset': in_data_id,
                'groupBy': ["EPARegion"],
                'statistics': [{
                    'type': "count",
                    'field': "ObjectId",
                    'outField': "objectid_count"
                }],
                'materialize': False
            },
            'outDataset': out_dataset
        }],
        'outDataset': out_dataset
    },
    'name': out_dataset,
    'fields': {
        "objectid_count": {
            'alias': "Count of ObjectId"
        },
        "EPARegion": {}
    }
}

In [ ]:
# Create the chart card
props['pages'][0]['cards'].append({
    'title': 'Card 2',
    'type': 'chart',
    'content': {
        'layers': [{
            'datasetId': out_dataset,
            'chartLines': {
                'mean': True
            },
            'colors': {}
        }],
        'type': 'column'
    }
})
# Place it to the right of any existing cards
props['pages'][0]['layout'].append({
    "x": 21,
    "y": 0,
    "w": 20,
    "h": 20
})

In [ ]:
# Save again
props["id"] = workspace_id
props["owner"] = gis.users.me.username
props["name"] = workbook_id
props["url"] = workspace_url
post_data = {
    'f': 'json',
    'title': title,
    'text': json.dumps(props)}
update_url = gis._url.lower() + '/sharing/rest/content/users/' + \
    gis.users.me.username + '/items/' + \
    workspace_id + '/update'
gis._portal.con.post(update_url, post_data)

# Working with InsightsWorkbook class

## Creating new workbook

In [ ]:
from insightsworkbook import InsightsWorkbook
workbook = InsightsWorkbook.new(gis, 'InsightsWorkbook() Test')

## Open existing workbook

In [ ]:
wb = gis.content.search('InsightsWorkbook', 'Insights Workbook')[0]
wb

In [ ]:
workbook = InsightsWorkbook.open(wb)

## Add feature layer & save

In [ ]:
lyr = gis.content.search('title:"Air Quality Monitors" AND owner:"my_username"', 'Feature Layer')[0]
new_dataset = workbook.add_feature_layer(lyr)
workbook.save()

## Add map

In [ ]:
workbook.add_map(new_dataset)
workbook.save()

## Add charts

In [ ]:
workbook.add_chart('column', new_dataset, 'EPARegion', 'esriFieldTypeString',
                   'count', 'ObjectId', 'esriFieldTypeInteger')
workbook.add_chart('bar', new_dataset, 'Status', 'esriFieldTypeString',
                   'avg', 'Elevation', 'esriFieldTypeDouble')
workbook.save()

## Update/refresh data

In [ ]:
from arcgis.features import FeatureLayerCollection
flayer_collection = FeatureLayerCollection.fromitem(lyr)
flayer_collection.manager.overwrite(r'\path\to\Air Quality Monitors.csv')

In [ ]:
workbook.update_dataset(lyr)
workbook.save()